<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/80_maritimes_address_pts_hex_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [1]:
%%capture
!pip install geohexgrid

In [2]:
import math
import geopandas as gpd
import geohexgrid as ghg
import pooch

In [ ]:
address_pts_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/MaritimesAddressPointDensity.gpkg.zip',
    known_hash='65f0bd9c3920d37f9540b0f903a47a9f65692ff8f0f6d9d22877a664fb62712f',
    processor=pooch.Unzip()
)[0]

In [ ]:
cities_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/CAN_adm.zip',
    known_hash='714b207c5673a5aa7d5114731701f28109351cbf54e12228714d427c7ef7a68a'
)

In [5]:
address_pts = gpd.read_file(
    address_pts_file,
    engine='pyogrio',
    layer='maritimes_address_pts'
)

cities = gpd.read_file(
    cities_file,
    engine='pyogrio',
    layer='CAN_adm3'
).to_crs(address_pts.crs)

In [6]:
def get_circumradius(target_area): # target_area in km2
    return (target_area * 1e6 / (3 * math.sqrt(3) / 2)) ** 0.5

hex_grid = ghg.make_grid_from_gdf(address_pts, R=get_circumradius(10.5))

In [7]:
# Calculate the Counts of addresses in each Hex grid
joined = gpd.sjoin(address_pts, hex_grid, predicate='within', how='left')
point_counts = joined.groupby('index_right').size().rename('Count')
hex_grid = hex_grid.join(point_counts, how='left')
hex_grid.to_file('hex_grid.gpkg')

In [8]:
(
    gpd.sjoin(
        hex_grid[['geometry', 'Count']],
        cities[['geometry', 'NAME_1', 'NAME_3']],
        predicate='intersects',
    )
    .rename(columns={'NAME_1': 'Province', 'NAME_3': 'City'})
    .sort_values('Count', ascending=False)
    .head(20)
    [['Province', 'City', 'Count']]
    .reset_index(drop=True)
)

,Province,City,Count
0,Nova Scotia,Halifax,7726
1,Nova Scotia,Halifax,4729
2,Nova Scotia,Halifax,3977
3,Nova Scotia,Halifax,3958
4,Nova Scotia,Halifax,3938
5,Nova Scotia,Halifax,3905
6,New Brunswick,Moncton City,3730
7,Nova Scotia,Halifax,3502
8,New Brunswick,Moncton City,3475
9,Nova Scotia,Amherst,3159


In [9]:
print(f'Total number of hexagons: {len(hex_grid):,}')
print(f'Total number of address points: {len(address_pts):,}')
print(f'Mean number of address points per hexagon: {hex_grid.Count.mean():.2f}')
print(f'Median number of address points per hexagon: {hex_grid.Count.median():.2f}')
print(f'Std. Dev of address points per hexagon: {hex_grid.Count.std():.2f}')

Total number of hexagons: 13,913
Total number of address points: 939,203
Mean number of address points per hexagon: 67.51
Median number of address points per hexagon: 20.00
Std. Dev of address points per hexagon: 225.05
